# 4 УРОК

ТИПЫ ДАННЫХ И ИХ ПРЕОБРАЗОВАНИЕ

## ШАГ 3
### Далее в заданиях используются данные с платформы Airbnb в Берлине ✈️.

Note: Как зайти в ClickHouse. Если таблицы не видны в списке – попробуйте выйти и зайти ещё раз.

Имеются следующие таблицы:

1. listings – информация о жилье, включая полные описания, характеристики и средние оценки в отзывах; поскольку столбцов очень много, нужные перечислены в текстах самих задач

2. calendar_summary – информация о доступности и цене того или иного жилья по дням

listing_id – идентификатор жилья (объявления)

date – дата

available – доступность жилья в данный день (t/f)

price – цена за ночь

3. reviews – отзывы

listing_id –  идентификатор жилья  

id – id отзыва

date – дата отзыва

reviewer_id – id ревьюера (автора отзыва)

reviewer_name – имя автора

comments – сам отзыв

## ШАГ 4
На Airbnb есть две основные группы пользователей:

Хозяева (хосты, hosts) – сдают жилье

Путешественники (travelers) – снимают

Начнем с анализа характеристик хозяев в таблице listings! 

Пользователи, сдающие квартиры на Airbnb, зарегистрировались в разное время. Кто-то – очень давно, а кто-то совсем недавно.

Давайте проверим, в какой месяц и год зарегистрировалось наибольшее количество новых хостов. В качестве ответа введите дату 

следующего формата: 2010-12

host_id – идентификатор хозяина

host_since – дата регистрации как хост

Note: Сам хост может встретиться в таблице несколько раз, если сдает несколько помещений, поэтому не забудьте оставить

уникальные значения host_id. Также обратите внимание на тип данных в host_since, возможно вам пригодится toStartOfMonth() для

извлечения части даты и toDateOrNull() для обработки пустых значений.

In [ ]:
SELECT COUNT(DISTINCT(host_id)) AS COUNT_HOST_id,
        toStartOfMonth(toDateOrNull(host_since)) AS Houst_since
       

FROM listings
GROUP BY Houst_since

ORDER BY COUNT_HOST_id DESC
LIMIT 10

525	2015-07-01
494	2015-08-01
403	2014-08-01
403	2014-07-01
377	2015-06-01
363	2014-09-01
356	2015-09-01
350	2015-05-01
328	2015-12-01
327	2016-03-01

#### ОТВЕТ : 2015-07

## ШАГ 5
Посмотрим на среднюю частоту ответа среди хозяев (f) и суперхозяев (t).

Значения частоты ответа хранятся как строки и включают значок %, который необходимо заменить на пустоту (''). 

После этого приведите столбец к нужному типу данных с помощью toInt32OrNull() и посчитайте среднюю частоту отклика в разбивке 

по тому, является ли хост суперхозяином или нет. В качестве ответа укажите наибольшее среднее.

host_response_rate – частота ответа

host_is_superhost – является ли суперхозяином

Важный момент: у каждого host_id есть только одно уникальное значение частоты ответа, а также одна единая отметка суперхозяина.

Чтобы посчитать показатели честно, нужно использовать подзапрос и взять уникальные комбинации с помощью DISTINCT. Примерная

структура финального запроса:

SELECT
    ... -- перечисляем все нужные колонки и операции
    
FROM (
    SELECT 
        DISTINCT column_1,
        column_2,
        column_3
    FROM 
        table_name) AS sub -- в подзапросе выбираем уникальные значения строк по интересующим колонкам
... -- не забываем LIMIT, сортировку и вот это вот всё


Пригодятся:

replaceAll(столбец, что_заменить, на_что_заменить) – замена значений в колонке

toIntOrNull(столбец)

Суперхозяева — опытные хозяева с отличными отзывами. В задаче используется таблица listings.

Подсказки

ВАЖНОЕ ЗАМЕЧАНИЕ!

Если вы делаете задание через REDASH, убедитесь, что в ответ вносите целое число 

( 75.00 -> 75 )

In [ ]:
SELECT
    host_is_superhost,
    AVG(toInt32OrNull(replaceAll(host_response_rate, '%', ''))) as AvgResp
FROM
    (SELECT DISTINCT(host_id) AS host_id,
        host_is_superhost AS host_is_superhost,
        host_response_rate AS host_response_rate
        

    FROM listings
    
    )
GROUP BY
    host_is_superhost


	
t	98
f	89.39845065678679

##### ОТВЕТ: 98

## ШАГ 6

In [ ]:
# """
# Сгруппируйте данные из listings по хозяевам (host_id) и посчитайте, какую цену за ночь в среднем каждый из них устанавливает 
# (у одного хоста может быть несколько объявлений). Идентификаторы сдаваемого жилья объедините в отдельный массив. 
# Таблицу отсортируйте по убыванию средней цены и убыванию host_id (в таком порядке). 
# В качестве ответа укажите первый массив в результирующей таблице, состоящий более чем из двух id. Например:

# 2343477,6340,32324


# host_id – идентификатор хозяина
# id – идентификатор жилья
# price – цена за ночь в конкретном месте
# Пригодятся:

# groupArray(столбец) – составление массива
# replaceRegexpAll(столбец, что_заменить, на_что_заменить) – паттерн [$,] с использованием функции replaceRegexpAll() 
# чтобы избавиться от $ и запятых Пример того, как выглядят цены в колонке price:
# +----------+
# | Price    | 
# +----------+
# | $16.00   | 
# | $2,228.00|  
# +----------+
# toFloat32OrNull(столбец)
# Note: id нужно вставить без пробелов, через запятую. 

# """

In [ ]:
SELECT 
    host_id,
    groupArray(id) AS Array_id,
    AVG(toFloat64OrNull(replaceRegexpAll(price, '[$,]', ''))) as AvgPricePerHost

FROM listings
GROUP BY host_id
ORDER BY AvgPricePerHost DESC,
        host_id DESC
LIMIT 10

49896804	9651871	9000
28957697	28462345	8600
187655517	25757977,25759146,25802565,25802651,25802838,25802909,25803050,25803117,25803218,25803260	6000
98159395	26186816,26186866,26186925,26186990,26187036,26187118,26187200,26187278,26187361,26249881	6000
90556837	22586252	5000
86166545	14201780	5000
12318581	7024131	4240
155140624	21447280,21479879	4011.5
109992090	26294126	3000
14623546	2860420	2500

#### ОТВЕТ:
25757977,25759146,25802565,25802651,25802838,25802909,25803050,25803117,25803218,25803260

## ШАГ 7
задание 

Немного усложним предыдущую задачу, и посчитаем разницу между максимальной и минимальной установленной ценой у каждого хозяина.

В качестве ответа укажите идентификатор хоста, у которого разница оказалась наибольшей. 

host_id – идентификатор хозяина

id – идентификатор жилья

price – цена за ночь в конкретном месте
Пригодятся:
    

replaceRegexpAll(столбец, что_заменить, на_что_заменить) – паттерн [$,] чтобы избавиться от $ и запятых

toFloat32OrNull(столбец)

Note: не забудьте повторить операции с предыдущего шага, обратите внимание на сортировку.

Подсказки

ВАЖНОЕ ЗАМЕЧАНИЕ!

Если вы делаете задание через REDASH, убедитесь, что в ответ вносите значение без запятых между порядками числа 
( 9,424,234.05 -> 9424234.05 )

In [ ]:
SELECT 
    host_id,
    groupArray(id) AS Array_id,
    MAX(toFloat64OrNull(replaceRegexpAll(price, '[$,]', ''))) as MaxPricePerHost,
    MIN(toFloat64OrNull(replaceRegexpAll(price, '[$,]', ''))) as MinPricePerHost,
    MaxPricePerHost - MinPricePerHost AS difference
FROM listings 
GROUP BY host_id
ORDER BY difference DESC

LIMIT 10


155140624	21447280,21479879	8000	23	7977
3003519	9536912,16237757,16250536,16252509,16254081,16280687,19153150,22966047	2500	100	2400
4259960	810056,6408850,7239371,18586882	2228	14	2214
102979040	19961235,20808102,20810849,22075653,23003841,23176584,23180356,23302498,23522653,25821868,26549470,26549473,26549474,26549615,26606882,26606887	1875	85	1790
134939989	26836678,29684637	1500	120	1380
7479020	12162808,15256845,16543873	1250	79	1171
54942921	10641148,10641334,10642720,10643786,10822913,13861835,14288914,14290848,14479069,14479963,19341361,19730036,19732253,21279723,21286310,26447631,28003596,28004298	899	39	860
9887915	1904731,17479412,28095362	1000	160	840
909226	8800109,8957894,9287838,10043384,13593507,13666450,16799799,17223235,20871245	885	99	786
42671030	28772056,29021563,29022213,29022451,29022524,29022547	1000	255	745

###### ОТВЕТ : 155140624

## ШАГ 8
Теперь сгруппируйте данные по типу жилья и выведите средние значения цены за ночь, размера депозита и цены уборки. 

Обратите внимание на тип данных, наличие значка $ и запятых в больших суммах. 

Для какого типа жилья среднее значение залога наибольшее?


room_type – тип сдаваемого жилья 

price – цена за ночь

security_deposit – залог за сохранность имущества

cleaning_fee – плата за уборку

Пригодятся:

toFloat32OrNull(столбец)

replaceRegexpAll(столбец, что_заменить, на_что_заменить) – паттерн [$,] чтобы избавиться от $ и запятых

In [ ]:
SELECT 
    room_type,
    AVG(toFloat64OrNull(replaceRegexpAll(price, '[$,]', ''))) as AvgPricePerHost,
    AVG(toFloat64OrNull(replaceRegexpAll(security_deposit, '[$,]', ''))) as Avg_security_deposit,
    AVG(toFloat64OrNull(replaceRegexpAll(cleaning_fee, '[$,]', ''))) as Avg_cleaning_fee
   
FROM listings 
GROUP BY room_type
ORDER BY Avg_security_deposit  DESC

LIMIT 10

Entire home/apt	83.34890878567431	274.9199887703537	37.01574996923834
Private room	52.47910525403156	117.24949118046133	15.385309460031017
Shared room	51.564189189189186	103.6140350877193	9.188172043010752

###### Ответ: Entire home/apt

## ШАГ 9
Отлично! В каких частях города средняя стоимость за ночь является наиболее низкой? 

Сгруппируйте данные по neighbourhood_cleansed и посчитайте среднюю цену за ночь в каждом районе.

В качестве ответа введите название места, где средняя стоимость за ночь ниже всего.

price – цена за ночь

neighbourhood_cleansed – район/округ города

Пригодятся:

toFloat32OrNull(столбец)

replaceRegexpAll(столбец, что_заменить, на_что_заменить) – паттерн [$,] чтобы избавиться от $ и запятых


In [ ]:
SELECT 
    neighbourhood_cleansed,
    AVG(toFloat64OrNull(replaceRegexpAll(price, '[$,]', ''))) as AvgPricePerHost
FROM listings 
GROUP BY neighbourhood_cleansed
ORDER BY AvgPricePerHost  

LIMIT 10

Neu-Hohenschönhausen Süd	20.5
Kölln. Vorstadt/Spindlersf.	23.75
Marzahn-Mitte	26.857142857142858
Hellersdorf-Süd	28.2
Gropiusstadt	30.333333333333332
Friedrichsfelde Süd	30.391304347826086
West 4	33.75
Grünau	33.90909090909091
Ost 1	34.270833333333336
Lankwitz	36.38461538461539

###### Ответ: Neu-Hohenschönhausen Süd

## ШАГ 10
В каких районах Берлина средняя площадь жилья, которое сдаётся целиком, является наибольшей? 

Отсортируйте по среднему и выберите топ-3. 


neighbourhood_cleansed – район

square_feet – площадь

room_type – тип (нужный – 'Entire home/apt')

Пригодится:

toFloat32OrNull(столбец)

In [ ]:
SELECT 
    neighbourhood_cleansed,
    AVG(toFloat64OrNull(square_feet)) as Avgsquare_feet
FROM listings 
WHERE room_type = 'Entire home/apt'
GROUP BY neighbourhood_cleansed

ORDER BY Avgsquare_feet DESC  

LIMIT 10


Kaulsdorf	2260
Schöneberg-Süd	1752.6666666666667
Lichtenrade	1237.5
Baumschulenweg	1184
Regierungsviertel	1123
Schmöckwitz/Karolinenhof/Rauchfangswerder	977
Brunnenstr. Nord	932.6666666666666
Kurfürstendamm	915
West 5	861
Friedrichsfelde Nord	807

###### ОТВЕТ : 
    Kaulsdorf
    Schöneberg-Süd
    Lichtenrade

## ШАГ 11

Напоследок давайте посмотрим, какая из представленных комнат расположена ближе всего к центру города. 

В качестве ответа укажите id объявления.

id – идентификатор жилья (объявления)

room_type – тип жилья ('Private room')

latitude – широта

longitude – долгота

52.5200 с.ш., 13.4050 в.д – координаты центра Берлина

Пригодятся:

toFloat64OrNull(столбец)
geoDistance(13.4050, 52.5200, долгота, широта)

In [ ]:
SELECT 
    id,
    geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) AS disance_to_centr
    
FROM listings 
WHERE room_type = 'Private room'


ORDER BY disance_to_centr

LIMIT 10

In [ ]:
19765058	58.80182
22543260	75.29827
1608059	95.993774
28518348	101.384964
27685334	109.362976
28477318	111.97329
25969985	113.42109
27516513	122.70006
23867497	130.00731
14167097	141.02852

###### ОТВЕТ : 19765058
это же 58 метров )) от центра города и всего 55$